# This notebook implements Deep learning techniques for a particle-in-halo classification framework. 

Reference with random forest: [ChJazhiel notebook](https://github.com/ChJazhiel/ML_ICF/blob/master/RF_Particles_z23.ipynb)

In [12]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from matplotlib.ticker import AutoMinorLocator

%matplotlib inline

# workdirectory = '/home/jazhiel/ML_Notebooks/Cosmology_ML/'
workdirectory = ''

## Import our dataset 

In [2]:
data_dict = np.load(workdirectory + 'OUTFILE1M.npz') #np.load('/path/to/nbody/outfile.npz')
test_flags  = data_dict['test_flags'] ## not important
test_hosts  = data_dict['test_hosts'] ### somewhat relevant
test_mass   = data_dict['test_mass'] ## important
test_labels = data_dict['test_labels'] ## important
test_input  = data_dict['test_input'] ## very important
#test_snid   = dict_data['test_snid']
#test_labels = dict_data['test_labels']
print(test_mass) ## Here I want to check how is the halo mass matrix composed of, the -1 means the halo 
#is not in our range of 10^12-10^13 M_sun
print(np.sum(test_labels)) ## Here I want to check how many label "1" do we have

[-1.00000000e+00  2.72242898e+13  1.13894322e+13 ... -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00]
289964


## Data preprocessing

In [3]:
### here we create or 10 vector dataset, I wonder if by adding some other information the classification will be better
### adding mass is not helpful, the classifier is perfect in that regard
dr1 = pd.DataFrame(test_input[0], columns = ['dr1'])
dr2 = pd.DataFrame(test_input[1], columns = ['dr2'])
dr3 = pd.DataFrame(test_input[2], columns = ['dr3'])
dr4 = pd.DataFrame(test_input[3], columns = ['dr4'])
dr5 = pd.DataFrame(test_input[4], columns = ['dr5'])
dr6 = pd.DataFrame(test_input[5], columns = ['dr6'])
dr7 = pd.DataFrame(test_input[6], columns = ['dr7'])
dr8 = pd.DataFrame(test_input[7], columns = ['dr8'])
dr9 = pd.DataFrame(test_input[8], columns = ['dr9'])
dr10 = pd.DataFrame(test_input[9], columns = ['dr10'])
#mass = pd.DataFrame(test_mass, columns = ['Halo_Mass'])
lbl = pd.DataFrame(test_labels, columns =['labels'])

## Select all features and create dataframe

In [4]:
df = pd.concat([dr1, dr2, dr3, dr4, dr5, dr6, dr7, dr8, dr9, dr10, lbl], axis=1, ignore_index=False, sort=False)
df

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
0,-0.209889,-0.187730,-0.169542,-0.155964,-0.100744,-0.049741,-0.011099,-0.010984,-0.002834,0.024089,0
1,0.364046,0.430008,0.453121,0.450021,0.374578,0.309706,0.240559,0.190503,0.174401,0.176335,1
2,0.311775,0.254511,0.227109,0.215286,0.148460,0.112097,0.090454,0.058546,0.035873,0.043613,1
3,0.033877,0.051836,0.065151,0.073930,0.100935,0.100271,0.094132,0.080007,0.059981,0.032160,0
4,-0.355428,-0.330448,-0.306125,-0.291701,-0.253399,-0.215256,-0.163749,-0.124680,-0.080408,-0.051999,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,0.022223,0.034349,0.046298,0.047812,0.027945,0.018082,0.012005,0.015623,0.028762,0.036779,0
999996,-0.178963,-0.188312,-0.189045,-0.187326,-0.182576,-0.170714,-0.155041,-0.143078,-0.136640,-0.120222,0
999997,-0.164632,-0.113120,-0.076089,-0.061724,0.009549,0.039809,0.057906,0.041327,0.010610,-0.020231,0
999998,-0.039698,0.063765,0.118363,0.140928,0.152761,0.125845,0.114529,0.113929,0.107155,0.098008,0


## Here we select a dataframe consisting of evenly separated labels

(I'm not sure if selecting all particles will impact in a different result)


In [5]:
## Sorting out the labels consisting in label '0' and label '1'
## Then we sample them in order to not selecting them in a specific range or shape

df_0 = df.sort_values('labels').head(710036).sample(289000)
df_1 = df.sort_values('labels').tail(289964).sample(289000) 
df_1.labels.sum()
df_r = pd.concat([df_0, df_1])
#df_scaled = scaler.fit_transform(df_r.drop(['labels'], axis = 1))  
#df_copy1 = pd.DataFrame(df_scaled, columns=['dr1', 'dr2', 'dr3', 'dr4', 'dr5', 'dr6', 'dr7', 'dr8', 'dr9', 'dr10'])

shuffle_df = df_r.sample(frac = 1.0)
#shuffle_df.insert(10, "Model", ['LC','SF']*14300, True)
shuffle_df

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
852894,0.102846,0.084565,0.074973,0.072405,0.089686,0.103981,0.105024,0.085952,0.051139,0.039769,1
446794,0.673512,0.456902,0.370605,0.340619,0.225556,0.161210,0.083929,0.038879,0.005992,-0.016802,1
767258,0.356473,0.214281,0.158017,0.136651,0.072421,0.052794,0.044656,0.043945,0.042434,0.040661,1
474997,0.257582,0.163283,0.141332,0.138210,0.134389,0.135707,0.122718,0.073848,0.002087,-0.057583,1
711891,0.173705,0.160352,0.143946,0.133133,0.135115,0.120767,0.082296,0.065740,0.054758,0.036958,0
...,...,...,...,...,...,...,...,...,...,...,...
266780,-0.134234,-0.066986,-0.034929,-0.024988,0.026008,0.038451,0.046804,0.069830,0.095081,0.098712,0
230973,-0.069222,-0.060010,-0.053405,-0.046040,-0.026596,-0.012115,0.021917,0.034834,0.039080,0.026853,1
272647,0.233213,0.170339,0.142273,0.132676,0.102715,0.079554,0.050270,0.018579,-0.020370,-0.047505,0
850627,-0.182761,-0.068712,-0.004495,0.009333,0.053104,0.059387,0.051115,0.045354,0.049470,0.046791,1


## define a size for our traning dataset 

I think that for 500k + particles we can divide into train, test and validation datasets

In [6]:
# Define a size for your train set 
train_size = int(0.8 * len(df_r))

# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

train_set

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
852894,0.102846,0.084565,0.074973,0.072405,0.089686,0.103981,0.105024,0.085952,0.051139,0.039769,1
446794,0.673512,0.456902,0.370605,0.340619,0.225556,0.161210,0.083929,0.038879,0.005992,-0.016802,1
767258,0.356473,0.214281,0.158017,0.136651,0.072421,0.052794,0.044656,0.043945,0.042434,0.040661,1
474997,0.257582,0.163283,0.141332,0.138210,0.134389,0.135707,0.122718,0.073848,0.002087,-0.057583,1
711891,0.173705,0.160352,0.143946,0.133133,0.135115,0.120767,0.082296,0.065740,0.054758,0.036958,0
...,...,...,...,...,...,...,...,...,...,...,...
268495,0.193962,0.233838,0.247723,0.250165,0.194097,0.158222,0.135334,0.106480,0.063084,0.027466,1
599640,0.243180,0.210116,0.177020,0.155800,0.164132,0.167751,0.149791,0.129122,0.104378,0.090101,0
74199,-0.079473,-0.118908,-0.130251,-0.132316,-0.123047,-0.108568,-0.084090,-0.071873,-0.050527,-0.040226,0
111717,0.263636,0.151838,0.113359,0.104937,0.030118,0.014838,0.023553,0.040797,0.057181,0.065337,1


## Select data, X for attributes, y for labels

In [7]:
X = shuffle_df.drop(['labels'], axis = 1)
ylabels = shuffle_df.labels
#ymodel = shuffle_df.Model
#ymodel = copy1.Model
X_train = train_set.drop(['labels'], axis = 1)
X_test = test_set.drop(['labels'], axis= 1)
y_trainl = train_set.labels
y_testl = test_set.labels
#y_trainM = train_set.Model
#y_testM = test_set.Model
print(X_train.shape)

(462400, 10)


## For the Confusion matrix 


In [8]:
#### THIS CELL IS FOR CALCULATE THE CONFUSION MATRIX ####



def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
    


np.set_printoptions(precision=2)

## Random forest with sklearn 

In [13]:
rf=RandomForestClassifier(n_estimators=100, max_depth=8, n_jobs=-1,
                          criterion = 'entropy', class_weight='balanced')

## n_jobs = -1 tells my computer to use all its cores, I have only 2, so it runs on 2 cores
#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(X_train,y_trainl)

ypred = rf.predict(X_test)
#####

print('Traning and Testing on raw data, all features \n');
#### Model accuracy
print("Accuracy:", metrics.accuracy_score(y_testl, ypred))

for i, score_forest in enumerate(cross_val_score(rf, X, ylabels, cv = 3)):
    print('Random Forest accuracy for the %d score: %0.2f' % (i, score_forest))
score_forest=cross_val_score(rf, X, ylabels, cv=3)
#score_tree
cv_scores = []
print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (score_forest.mean(), score_forest.std() * 2 ))
cv_score = score_forest.mean()
cv_scores.append(cv_score)

### We perform a cross validation score to see how accurate our decision tree is by 
### splitting the dataset into 10 folds for training and testing

Traning and Testing on raw data, all features 

Accuracy: 0.7839273356401384
Random Forest accuracy for the 0 score: 0.78
Random Forest accuracy for the 1 score: 0.79
Random Forest accuracy for the 2 score: 0.78
Random Forest Accuracy: 0.78 (+/- 0.00)


## Perceptron with Keras

In [ ]:
#para scikit-learn: (samples,features)
#test_input_T = test_input.T
#X_train, X_test, y_train, y_test = train_test_split(test_input_T,test_labels,test_size=0.25,random_state=None)

#ANN
test1_model = models.Sequential()
test1_model.add(layers.Dense(100,activation='relu',input_shape=(10,)))
test1_model.add(layers.Dense(100,activation='relu'))
#test1_model.add(layers.Dense(32,activation='relu'))
#test1_model.add(layers.Dense(10,activation='relu'))
test1_model.add(layers.Dense(1, activation='sigmoid'))

test1_model.compile(optimizer=optimizers.Adam(lr=0.0001),#RMSprop(lr=0.001),
                    loss=losses.binary_crossentropy,
                    metrics=['accuracy'])

test1_model.summary()

test1_model_history = test1_model.fit(X_train,
                                      y_trainl,
                                      epochs=50,
                                      batch_size=32,
                                      validation_data=(X_test,y_testl),
                                      verbose=1)

#plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(11,4.5))    

results_dict = test1_model_history.history
epochs = range(1,len(results_dict['accuracy'])+1)

#accuracy
acc_values = results_dict['accuracy']
val_acc_values = results_dict['val_accuracy']
    
axs[0].plot(epochs,acc_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[0].plot(epochs,val_acc_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[0].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[0].xaxis.set_minor_locator(minorLocatorX)
axs[0].set_ylabel('accuracy')
axs[0].set_ylim([0.5,1.02])   
minorLocatorY = AutoMinorLocator()
axs[0].yaxis.set_minor_locator(minorLocatorY)
axs[0].tick_params(which='major', length=6)
axs[0].tick_params(which='minor', length=3, color='black')       
axs[0].legend(loc='lower right')
    
#loss
loss_values = results_dict['loss']
val_loss_values = results_dict['val_loss']
       
axs[1].plot(epochs,loss_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[1].plot(epochs,val_loss_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[1].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[1].xaxis.set_minor_locator(minorLocatorX)   
axs[1].set_ylabel('loss')
minorLocatorY = AutoMinorLocator()
axs[1].yaxis.set_minor_locator(minorLocatorY)
axs[1].tick_params(which='major', length=6)
axs[1].tick_params(which='minor', length=3, color='black')    
axs[1].legend(loc='upper right')

axs[0].set_title('Training and test accuracy after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),acc_values[-1],val_acc_values[-1]));

axs[1].set_title('Training and test loss after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),loss_values[-1],val_loss_values[-1]));

2022-05-09 22:52:37.139653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-09 22:52:37.139670: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-09 22:52:37.139682: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (isidro-pc): /proc/driver/nvidia/version does not exist
2022-05-09 22:52:37.139928: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/isidro/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 11,301
Trainable params: 11,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
 1225/14450 [=>............................] - ETA: 7s - loss: 0.5117 - accuracy: 0.7649